# Notebook para explorar datos y definir acciones a ejecutar en transform.py

In [28]:
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)

# Cargar dfs
df_bibliotecas_path = '../data/bibliotecas/2022-octubre/bibliotecas-07-10-2022.csv'
df_cines_path = '../data/cines/2022-octubre/cines-07-10-2022.csv'
df_museos_path = '../data/museos/2022-octubre/museos-07-10-2022.csv'

bibliotecas_df = pd.read_csv(df_bibliotecas_path, encoding='utf8')
cines_df = pd.read_csv(df_cines_path, encoding='utf8')
museos_df = pd.read_csv(df_museos_path, encoding='utf8')

# Ver tamaños y columnas de cada uno
dfs = [bibliotecas_df, cines_df, museos_df]
for df in dfs:
    display(df.columns)
    display(df.shape)

Index(['Cod_Loc', 'IdProvincia', 'IdDepartamento', 'Observacion', 'Categoría',
       'Subcategoria', 'Provincia', 'Departamento', 'Localidad', 'Nombre',
       'Domicilio', 'Piso', 'CP', 'Cod_tel', 'Teléfono', 'Mail', 'Web',
       'Información adicional', 'Latitud', 'Longitud', 'TipoLatitudLongitud',
       'Fuente', 'Tipo_gestion', 'año_inicio', 'Año_actualizacion'],
      dtype='object')

(2017, 25)

Index(['Cod_Loc', 'IdProvincia', 'IdDepartamento', 'Observaciones',
       'Categoría', 'Provincia', 'Departamento', 'Localidad', 'Nombre',
       'Dirección', 'Piso', 'CP', 'cod_area', 'Teléfono', 'Mail', 'Web',
       'Información adicional', 'Latitud', 'Longitud', 'TipoLatitudLongitud',
       'Fuente', 'tipo_gestion', 'Pantallas', 'Butacas', 'espacio_INCAA',
       'año_actualizacion'],
      dtype='object')

(329, 26)

Index(['Cod_Loc', 'IdProvincia', 'IdDepartamento', 'Observaciones',
       'categoria', 'subcategoria', 'provincia', 'localidad', 'nombre',
       'direccion', 'piso', 'CP', 'cod_area', 'telefono', 'Mail', 'Web',
       'Latitud', 'Longitud', 'TipoLatitudLongitud', 'Info_adicional',
       'fuente', 'jurisdiccion', 'año_inauguracion', 'actualizacion'],
      dtype='object')

(1182, 24)

## Transformaciones para master dataset
### 1° Unificar nombres de columnas comunes requeridas entre los 3 dfs

In [29]:
# Columnas para master_df: cod_localidad, id_provincia, id_departamento, categoría, 
# provincia, localidad, nombre, domicilio, código postal, número de teléfono, mail, web

# Cargar nombres de columnas de interés de cada df en mismo orden
columns = ['Cod_Loc', 'IdProvincia', 'IdDepartamento', 'Categoría', 'Provincia', 
            'Localidad','Nombre','Domicilio', 'CP', 'Cod_tel', 'Teléfono', 'Mail', 'Web']
bibliotecas_columns = ['Cod_Loc', 'IdProvincia', 'IdDepartamento', 'Categoría', 
                        'Provincia', 'Localidad','Nombre','Domicilio', 'CP', 'Cod_tel', 
                        'Teléfono', 'Mail', 'Web']
cines_columns = ['Cod_Loc', 'IdProvincia', 'IdDepartamento', 'Categoría', 'Provincia', 
                    'Localidad','Nombre','Dirección', 'CP', 'cod_area', 'Teléfono', 'Mail', 
                    'Web']
museos_columns = ['Cod_Loc', 'IdProvincia', 'IdDepartamento', 'categoria', 'provincia', 
                    'localidad','nombre','direccion', 'CP', 'cod_area', 'telefono', 
                    'Mail', 'Web' ]

# Renombrar columnas en cada df para que todos sean iguales y se pueda concatenar
bibliotecas_df.rename(columns=dict(zip(bibliotecas_columns, columns)), inplace=True)
cines_df.rename(columns=dict(zip(cines_columns, columns)), inplace=True)
museos_df.rename(columns=dict(zip(museos_columns, columns)), inplace=True)

# Concatenar dfs en master y ver resultados
master_df = pd.concat([bibliotecas_df[columns], cines_df[columns], museos_df[columns]])
display(master_df.columns, master_df.shape)
master_df.sample(5)

Index(['Cod_Loc', 'IdProvincia', 'IdDepartamento', 'Categoría', 'Provincia',
       'Localidad', 'Nombre', 'Domicilio', 'CP', 'Cod_tel', 'Teléfono', 'Mail',
       'Web'],
      dtype='object')

(3528, 13)

,Cod_Loc,IdProvincia,IdDepartamento,Categoría,Provincia,Localidad,Nombre,Domicilio,CP,Cod_tel,Teléfono,Mail,Web
1143,94007010,94,94007,Espacios de Exhibición Patrimonial,"Tierra del Fuego, Antártida e Islas del Atlántico Sur",Río Grande,Museo Fueguino De Arte,Av. Belgrano Nº 319,V9420BBA,2964.0,425 823,NaN,NaN
1067,6833080,6,6833,Bibliotecas Populares,Buenos Aires,San Francisco de Belloq,Biblioteca Popular San Francisco de Bellocq,Calle 15 446,7505.0,2982.0,497013.0,bibsanfrabe@eternet.cc,NaN
841,62042010,62,62042,Espacios de Exhibición Patrimonial,Río Negro,Allen,Museo Municipal De Allen,Libertad 250,R8328ASP,2941.0,453 403,museodeallen@yahoo.com.ar,NaN
988,86049110,86,86049,Bibliotecas Populares,Santiago del Estero,Santiago del Estero,Biblioteca Popular Jorge W. Abalos,Romualdo Helman 158,4200.0,385.0,4392109.0,bjwasantiago@yahoo.com.ar,NaN
15,46042010,46,46042,Salas de cine,La Rioja,Chilecito,Cinema,La Plata Y 19 De Febrero,5360,380.0,4626907.0,ericontadoralinares@gmail.com,http://cinema3dlarioja.com/


### 2° Unificar cod_tel y telefono en una única columna

In [30]:
display(master_df['Cod_tel'].dtype, master_df['Teléfono'].dtype)
master_df[['Cod_tel', 'Teléfono']].sample(9)

dtype('float64')

dtype('O')

,Cod_tel,Teléfono
205,362.0,443 9315
957,387.0,4220765.0
610,388.0,423 423
1004,3483.0,496 125
328,11.0,5411 4781
965,299.0,4439077.0
584,3455.0,492 191
1839,341.0,4663399.0
777,2473.0,480143.0


In [31]:
# Cod_tel es de tipo float por lo que los nros tienen agregado .0 al final, y hay valores nulos
# Teléfono es tipo str, tiene valores con espacios en blanco, pero también tiene algunos con .0 al final y valores nulos
# La nueva columna será str para preservar nulos, y se creará uniendo los dos campos (siempre y cuando haya valor en Teléfono)
# separados por un espacio (para seguir formato presentado), previa eliminación de los .0

def make_phone_number(row):

    if not pd.isnull(row['Teléfono']):
        # convierto a str
        tel = str(row['Teléfono']).strip()
        # elimino .0 final si tiene
        if tel.find('.0') != -1:
            tel = tel[:tel.find('.0')]
        # chequeo código para ver si debo concatenar o va solo nro de telefono sin cod area
        if not pd.isnull(row['Cod_tel']):
            cod = str(row['Cod_tel']).strip()
            if cod.find('.0') != -1:
                cod = cod[:cod.find('.0')]
            row['Nro_Teléfono'] = cod + ' ' + tel
        else:
            row['Nro_Teléfono'] = tel
    else:
    # si no tengo número de teléfono no me interesa si hay código de área 
        row['Nro_Teléfono'] = np.nan
    return row    

master_df = master_df.apply(make_phone_number, axis='columns')

master_df[['Cod_tel', 'Teléfono', 'Nro_Teléfono']].sample(12)
master_df.drop(['Cod_tel', 'Teléfono'], inplace=True, axis='columns')
columns.remove('Cod_tel')
columns.remove('Teléfono')

### Explorar datos por columna

In [32]:
# Explorar datos, chequeando nulos, valores únicos y distribución de valores
total_rows = master_df.shape[0]
for column in columns:
    display(f'---------------------------- Columna {column} - {master_df[column].dtype} ----------------------------')
    no_nulos = master_df[column].count()
    unique_values_count = len(master_df[column].unique())
    display(f'Total: {total_rows}  |  Nulos: {total_rows - no_nulos}  |  No nulos: {no_nulos} | Únicos: {unique_values_count}')
    if unique_values_count > 30:
        display(f'Top Distribución de valores:')
        display(master_df[column].value_counts().iloc[:15])
    else:
        display(f'Valores únicos:')
        display(master_df[column].unique())
        display(f'Distribución de valores:')
        display(master_df[column].value_counts())


'---------------------------- Columna Cod_Loc - int64 ----------------------------'

'Total: 3528  |  Nulos: 0  |  No nulos: 3528 | Únicos: 1197'

'Top Distribución de valores:'

2000010     210
14014010     92
82063170     72
6441030      63
82084270     61
66028050     50
22140060     46
58035070     37
86049110     31
50007010     31
70028010     30
90084010     27
6056010      24
6427010      22
38021060     21
Name: Cod_Loc, dtype: int64

'---------------------------- Columna IdProvincia - int64 ----------------------------'

...

s/d                                         561
rmazzutti@hoyts.com.ar                       11
iventura@cinemark.com.ar                      9
info@atlascines.com.ar                        5
fharari@hotmail.com.ar                        4
diversidad.cultural@santafeciudad.gov.ar      4
admin@lastipasboulevard.com.ar                3
nuevocineteatrorex@gmail.com                  3
cluduena@grupodinosaurio.com                  3
rabdenur@gmail.com                            3
ericontadoralinares@gmail.com                 3
naiarg@naiarg.com.ar                          3
gabriel.guralnik@gmail.com                    2
york.cultura@vicentelopez.gov.ar              2
museoesperanza@esperanza.gov.ar               2
Name: Mail, dtype: int64

'---------------------------- Columna Web - object ----------------------------'

'Total: 3528  |  Nulos: 3124  |  No nulos: 404 | Únicos: 308'

'Top Distribución de valores:'

http://www.cinemacenter.com.ar/            17
s/d                                        16
https://www.cinemarkhoyts.com.ar/          11
https://www.cinemarkhoyts.com.ar           10
https://www.villagecines.com/               8
cineatlasweb.com.ar                         7
https://www.multiplex.com.ar/               6
https://www.todoshowcase.com/               6
http://www.cinesunstar.com/                 6
www.ara.mil.ar                              4
www.enteculturaltucuman.gov.ar              4
http://cinema3dlarioja.com/                 3
http://www.cinemalaplata.com                3
www.culturarionegro.gov.ar                  3
https://www.cinesdinomall.com.ar/Inicio     2
Name: Web, dtype: int64

#### Resultados de de la exploración
- IdProvincia tiene 24 valores únicos pero Provincia tiene 27 valores únicos. Hay 3 provincias que tienen dos nombres cada una y se deben unificar: 
1. Neuquén y Neuquén\xa0, 
2. Tierra del Fuego y Tierra del Fuego, Antártida e Islas del Atlántico Sur
3. Santa Fe y Santa Fé
- Domicilio: tiene 18 nulos + 30 'sin dirección', convertir a nulos
- Web tiene 3124 nulos y 16 valores 's/d', convertir a nulos
- Mail tiene 918 nulos y 561 's/d', convertir a nulos
- CP es string pero tiene valores con .0 al final, eliminar .0

### 3° Unificar nombres de Provincia

In [33]:
master_df[['IdProvincia', 'Provincia', 'Categoría']].groupby(['IdProvincia', 'Provincia']).count().sort_values(by='IdProvincia')

def unify_province_names(row):
    if row['Provincia'] == 'Neuquén\xa0':
        row['Provincia'] = 'Neuquén'
    elif row['Provincia'] == 'Santa Fe':
        row['Provincia'] = 'Santa Fé'
    elif row['Provincia'] == 'Tierra del Fuego':
        row['Provincia'] = 'Tierra del Fuego, Antártida e Islas del Atlántico Sur'
    return row
master_df = master_df.apply(unify_province_names, axis='columns')

master_df[['IdProvincia', 'Provincia', 'Categoría']].groupby(['IdProvincia', 'Provincia']).count().sort_values(by='IdProvincia')

,,Categoría
IdProvincia,Provincia,
2,Ciudad Autónoma de Buenos Aires,210
6,Buenos Aires,801
10,Catamarca,74
14,Córdoba,346
18,Corrientes,80
22,Chaco,107
26,Chubut,102
30,Entre Ríos,128
34,Formosa,34


- Se detecta una row que tiene Salta con codigo 58, que corresponde a Neuquén. Cambiar a 66
### 4° Cambiar cód de Salta en row erronea

In [34]:
# Chequear si lo que está incorrecto es el codigo o el nombre de la pcia.
master_df.loc[(master_df['Provincia'] == 'Salta') & (master_df['IdProvincia'] == 58)]

,Cod_Loc,IdProvincia,IdDepartamento,Categoría,Provincia,Localidad,Nombre,Domicilio,CP,Mail,Web,Nro_Teléfono
860,58035070,58,58035,Espacios de Exhibición Patrimonial,Salta,Salta,Museo Catedralicio,Av. Belgrano 589,NaN,NaN,NaN,NaN


In [35]:
master_df.loc[(master_df['Provincia'] == 'Salta') & (master_df['IdProvincia'] == 58), 'IdProvincia'] = 66

#chequear con group by
master_df[['IdProvincia', 'Provincia', 'Categoría']].groupby(['IdProvincia', 'Provincia']).count().sort_values(by='IdProvincia')

,,Categoría
IdProvincia,Provincia,
2,Ciudad Autónoma de Buenos Aires,210
6,Buenos Aires,801
10,Catamarca,74
14,Córdoba,346
18,Corrientes,80
22,Chaco,107
26,Chubut,102
30,Entre Ríos,128
34,Formosa,34


- Antes de unificar nulos, chequear que no haya 0s o espacios en blanco en ninguna columna

In [36]:
from pandas.api.types import is_string_dtype, is_numeric_dtype

columns.append('Nro_Teléfono')

for column in columns:
    if is_string_dtype(master_df[column]):
        # Para str chequeo espacio en blanco y vacio
        empty_count = master_df.loc[master_df[column] == '', column].count()
        whitespace_count = master_df.loc[master_df[column] == ' ', column].count()        
        display(f'{column}: empty {empty_count}, whitespace {whitespace_count}')
    elif is_numeric_dtype(master_df[column]):
        # Para numeric chequeo 0
        zero_count = master_df.loc[master_df[column] == 0, column].count()
        display(f'{column}: zero {zero_count}')

'Cod_Loc: zero 0'

'IdProvincia: zero 0'

'IdDepartamento: zero 0'

'Categoría: empty 0, whitespace 0'

'Provincia: empty 0, whitespace 0'

...

'Domicilio: empty 0, whitespace 0'

'CP: empty 0, whitespace 0'

'Mail: empty 0, whitespace 0'

'Web: empty 0, whitespace 0'

'Nro_Teléfono: empty 0, whitespace 0'

### 5° Unificar nulos

In [37]:
#Domicilio: tiene 18 nulos + 30 'Sin dirección', convertir a nulos
#Web tiene 3124 nulos y 16 valores 's/d', convertir a nulos
#Mail tiene 918 nulos y 561 's/d', convertir a nulos

def convert_text_nulls_to_nan(row):
    nulls = ['sin dirección', 's/d']
    if str(row['Domicilio']).strip().lower() in nulls:
        row['Domicilio'] = np.nan
    if str(row['Web']).strip().lower() in nulls:
        row['Web'] = np.nan
    if str(row['Mail']).strip().lower() in nulls:
        row['Mail'] = np.nan
    return row

master_df = master_df.apply(convert_text_nulls_to_nan, axis='columns')
# Check
display(master_df['Domicilio'].value_counts())
display(master_df['Mail'].value_counts())
display(master_df['Web'].value_counts())



Sarmiento               10
San Martín               9
25 de mayo               5
9 de julio               5
Sarmiento 450            4
                        ..
Junín 2238               1
Chubut y O.del Sur       1
Ruta 36 Km 3             1
Av. José Roberti 363     1
Ernesto Padilla S/N      1
Name: Domicilio, Length: 3367, dtype: int64

rmazzutti@hoyts.com.ar                      11
iventura@cinemark.com.ar                     9
info@atlascines.com.ar                       5
diversidad.cultural@santafeciudad.gov.ar     4
fharari@hotmail.com.ar                       4
                                            ..
dirrecioncultura@52.coopenet.com.ar          1
biblio2945amenabar@hotmail.com               1
bibliotecahoracioquiroga@live.com            1
bibliotecalugones@bombal.net.ar              1
museoejercitotucuman@hotmail.com             1
Name: Mail, Length: 1978, dtype: int64

http://www.cinemacenter.com.ar/                                                       17
https://www.cinemarkhoyts.com.ar/                                                     11
https://www.cinemarkhoyts.com.ar                                                      10
https://www.villagecines.com/                                                          8
cineatlasweb.com.ar                                                                    7
                                                                                      ..
https://twitter.com/cineteatroalfa                                                     1
https://www.facebook.com/pg/CineAHZ/about/?ref=page_internal                           1
https://www.facebook.com/pg/Cines-Avenida-112830095434025/about/?ref=page_internal     1
https://www.facebook.com/cine.italia/                                                  1
www.casaindependencia.com.ar                                                           1
Name: Web, Length: 30

### 6° Quitar .0 de CP

In [38]:
display(master_df['CP'].dtype)
master_df['CP'].sample(6)

dtype('O')

1552         NaN
1038        2000
1555      1744.0
351     C1103ACH
196         3700
33           NaN
Name: CP, dtype: object

In [42]:
master_df['CP'] = master_df['CP'].map(lambda x: str(x)[:-2] if str(x).find('.0') != -1 else x)
master_df['CP'].sample(6)

826         5248
589         3100
74          7111
1922        5109
457     X5850CBA
505          NaN
Name: CP, dtype: object

## 